<p align="center">
  <img src="R_container.png" />
</p>


# Introduction
This Kubeflow notebook will first ensure we can connect to the Kubeflow Pipeline Client. Then, it will define a container component called *generate_quote*, create a pipeline called quote_pipeline, and lastly submit the pipeline to the KFP SDK. We will then be able to see the pipeline run and associated artifacts within the Kubeflow Central Dashboard. Run each cell below in order and to view the results.

In [2]:
import kfp
from kfp import dsl
from kfp import compiler
from kfp.client import Client


client = kfp.Client()
print(client.list_experiments())

{'experiments': [{'created_at': datetime.datetime(2024, 2, 17, 17, 59, 4, tzinfo=tzlocal()),
                  'description': 'my first Kubeflow experiment! ',
                  'display_name': 'hello_kubeflow',
                  'experiment_id': 'c9e56f11-3433-4cef-a1a4-d3fb20c9b26e',
                  'namespace': 'christensenc3526',
                  'storage_state': 'AVAILABLE'},
                 {'created_at': datetime.datetime(2024, 2, 18, 16, 5, 37, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'new_experiment',
                  'experiment_id': 'bc7afbde-6c81-41e5-b7ef-ef81545c36e7',
                  'namespace': 'christensenc3526',
                  'storage_state': 'AVAILABLE'},
                 {'created_at': datetime.datetime(2024, 2, 26, 4, 33, 34, tzinfo=tzlocal()),
                  'description': None,
                  'display_name': 'Default',
                  'experiment_id': '7671697d-de7a-47a8-975a-27b0b89575ee',
  

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [14]:
@dsl.container_component
def generate_quote(total_cost: float, state_tax: float, quote: dsl.OutputPath(str)):
    """output a quote"""
    return dsl.ContainerSpec(
        image='chasechristensen/generate_quote:v1',
        command=[
            "Rscript","/usr/local/bin/generate_quote.r"
        ],
        args=[total_cost,state_tax,quote])

In [15]:
@dsl.pipeline
def quote_pipeline(x: float, y: float) -> str:
    task1 = generate_quote(total_cost=x, state_tax=y)
    return task1.output


In [16]:
compiler.Compiler().compile(quote_pipeline, 'pipeline.yml')

In [17]:
run = client.create_run_from_pipeline_package(
    'pipeline.yml',
    arguments={
        'x': 500.0,
        'y':0.35
    },
)